# Explore the AUG workflow and single flux tube UQ data sets

- this uses 
  - uq/data/AUG_mix-lim_gem_inoutput.txt for the WF data
    - with limits on Te, dTe, Ti and dTi
  - uq/data/gem_uq_inoutput.csv for the single flux tube UQ data

## A fairly standard set of imports

In [ ]:
%matplotlib inline
#%matplotlib notebook
import os
import matplotlib
if not os.getenv("DISPLAY"): matplotlib.use('Agg')
import matplotlib.pylab as plt
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')
import numpy as np
import pandas as pd
pd.options.display.max_rows = 100
from utility import _ewma, plot_smoothed, plot_unsmoothed

## Read and examine the workflow data

In [ ]:
# read the data from the AUG workflow runs
AUG_gem1 = pd.read_table('../../data/AUG_gem_inoutput.txt', delimiter='  *', engine='python')
AUG_gem2 = pd.read_table('../../data/AUG_mix-lim_gem_inoutput.txt', delimiter='  *', engine='python') 

In [ ]:
AUG_gem1.describe().T

In [ ]:
AUG_gem2.describe().T

In [ ]:
AUG_tmp = pd.read_table('../../data/AUG_mix-lim_flux_area.txt', delimiter='  *', engine='python')
AUG_tmp.describe().T



### Set up some column selections

In [ ]:
active_t = []
active_t.append(['time', 'Te-ft1', 'dTe-ft1', 'Ti-ft1', 'dTi-ft1', 'flux-Te-ft1', 'flux-Ti-ft1'])
active_t.append(['time', 'Te-ft2', 'dTe-ft2', 'Ti-ft2', 'dTi-ft2', 'flux-Te-ft2', 'flux-Ti-ft2'])
active_t.append(['time', 'Te-ft3', 'dTe-ft3', 'Ti-ft3', 'dTi-ft3', 'flux-Te-ft3', 'flux-Ti-ft3'])
active_t.append(['time', 'Te-ft4', 'dTe-ft4', 'Ti-ft4', 'dTi-ft4', 'flux-Te-ft4', 'flux-Ti-ft4'])
active_t.append(['time', 'Te-ft5', 'dTe-ft5', 'Ti-ft5', 'dTi-ft5', 'flux-Te-ft5', 'flux-Ti-ft5'])
active_t.append(['time', 'Te-ft6', 'dTe-ft6', 'Ti-ft6', 'dTi-ft6', 'flux-Te-ft6', 'flux-Ti-ft6'])
active_t.append(['time', 'Te-ft7', 'dTe-ft7', 'Ti-ft7', 'dTi-ft7', 'flux-Te-ft7', 'flux-Ti-ft7'])
active_t.append(['time', 'Te-ft8', 'dTe-ft8', 'Ti-ft8', 'dTi-ft8', 'flux-Te-ft8', 'flux-Ti-ft8'])

## Read and examine the UQ data

In [ ]:
# read the data from the AUG single flux tube UQ runs
G = pd.read_csv('../../data/gem_uq_inoutput.csv')
G.describe().T

### Set up some column selections

In [ ]:
# setup some column groups 
Te_cols = ['Te-ft1', 'Te-ft2', 'Te-ft3', 'Te-ft4', 'Te-ft5', 'Te-ft6', 'Te-ft7', 'Te-ft8']
Ti_cols = ['Ti-ft1', 'Ti-ft2', 'Ti-ft3', 'Ti-ft4', 'Ti-ft5', 'Ti-ft6', 'Ti-ft7', 'Ti-ft8']
dTe_cols = ['dTe-ft1', 'dTe-ft2', 'dTe-ft3', 'dTe-ft4', 'dTe-ft5', 'dTe-ft6', 'dTe-ft7', 'dTe-ft8']
dTi_cols = ['dTi-ft1', 'dTi-ft2', 'dTi-ft3', 'dTi-ft4', 'dTi-ft5', 'dTi-ft6', 'dTi-ft7', 'dTi-ft8']
Te_flux_cols = ['flux-Te-ft1', 'flux-Te-ft2', 'flux-Te-ft3', 'flux-Te-ft4', 
                'flux-Te-ft5', 'flux-Te-ft6', 'flux-Te-ft7', 'flux-Te-ft8'] 
Ti_flux_cols = ['flux-Ti-ft1', 'flux-Ti-ft2', 'flux-Ti-ft3', 'flux-Ti-ft4', 
                'flux-Ti-ft5', 'flux-Ti-ft6', 'flux-Ti-ft7', 'flux-Ti-ft8']
ne_cols = ['ne-ft1', 'ne-ft2', 'ne-ft3', 'ne-ft4', 'ne-ft5', 'ne-ft6', 'ne-ft7', 'ne-ft8']
dne_cols = ['dne-ft1', 'dne-ft2', 'dne-ft3', 'dne-ft4', 'dne-ft5', 'dne-ft6', 'dne-ft7', 'dne-ft8']
ne_flux_cols = ['flux-ne-ft1', 'flux-ne-ft2', 'flux-ne-ft3', 'flux-ne-ft4', 
                'flux-ne-ft5', 'flux-ne-ft6', 'flux-ne-ft7', 'flux-ne-ft8']

In [ ]:
# setup some column groups 
dvdrho_cols = ['dvdrho1', 'dvdrho2', 'dvdrho3', 'dvdrho4', 'dvdrho5', 'dvdrho6', 'dvdrho7', 'dvdrho8']
surfaceA_cols = ['surfaceA1', 'surfaceA2', 'surfaceA3', 'surfaceA4', 'surfaceA5', 'surfaceA6', 'surfaceA7', 'surfaceA8']
Te_source_int_cols = ['Te-source-int-ft1', 'Te-source-int-ft2', 'Te-source-int-ft3', 'Te-source-int-ft4', 
                'Te-source-int-ft5', 'Te-source-int-ft6', 'Te-source-int-ft7', 'Te-source-int-ft8'] 
Ti_source_int_cols = ['Ti-source-int-ft1', 'Ti-source-int-ft2', 'Ti-source-int-ft3', 'Ti-source-int-ft4', 
                'Ti-source-int-ft5', 'Ti-source-int-ft6', 'Ti-source-int-ft7', 'Ti-source-int-ft8']
E_source_int_cols = ['E-source-int-ft1', 'E-source-int-ft2', 'E-source-int-ft3', 'E-source-int-ft4', 
                'E-source-int-ft5', 'E-source-int-ft6', 'E-source-int-ft7', 'E-source-int-ft8']
E_flux_cols = ['flux-E-ft1', 'flux-E-ft2', 'flux-E-ft3', 'flux-E-ft4', 
               'flux-E-ft5', 'flux-E-ft6', 'flux-E-ft7', 'flux-E-ft8']

In [ ]:
AUG_tmp[E_source_int_cols] = np.array(AUG_tmp[Te_source_int_cols]) + np.array(AUG_tmp[Ti_source_int_cols])
AUG_tmp[Te_flux_cols] = np.array(AUG_tmp[Te_source_int_cols]) / np.array(AUG_tmp[dvdrho_cols])
AUG_tmp[Ti_flux_cols] = np.array(AUG_tmp[Ti_source_int_cols]) / np.array(AUG_tmp[dvdrho_cols])
AUG_tmp[E_flux_cols] = np.array(AUG_tmp[E_source_int_cols]) / np.array(AUG_tmp[dvdrho_cols])
AUG_gem2[E_flux_cols] = np.array(AUG_gem2[Te_flux_cols]) + np.array(AUG_gem2[Ti_flux_cols])

In [ ]:
AUG_tmp[E_flux_cols].describe().T

In [ ]:
fig, ax = plt.subplots(4, 2, figsize=(24,24))
AUG_tmp.plot(x='time', y=dvdrho_cols, ax=ax[0,0])
AUG_tmp.plot(x='time', y=surfaceA_cols, ax=ax[0,1])
AUG_tmp.plot(x='time', y=Te_source_int_cols, ax=ax[1,0])
AUG_tmp.plot(x='time', y=Ti_source_int_cols, ax=ax[1,1])
AUG_tmp.plot(x='time', y=Te_flux_cols, ax=ax[2,0])
AUG_tmp.plot(x='time', y=Ti_flux_cols, ax=ax[2,1])
AUG_tmp.plot(x='time', y=E_source_int_cols, ax=ax[3,0])
AUG_tmp.plot(x='time', y=E_flux_cols, ax=ax[3,1]);

In [ ]:
import importlib, utility
importlib.reload(utility)
from utility import _ewma, plot_smoothed, plot_unsmoothed

## Flux tube electron and ion energy fluxes

In [ ]:
plot_unsmoothed(
    [{"data": AUG_gem1, "style": '-.'}, {"data": AUG_gem2, "style": '-'}, {"data": AUG_tmp, "style": ':'}],
    [{"cols": Te_flux_cols, "label": 'electron energy flux density [W m^{-2}]', "limit": (1e2,1e6)},
     {"cols": Ti_flux_cols, "label": 'ion energy flux density [W m^{-2}]', "limit": (1e2,1e6)}],
    logy=True)

### Smoothed electron and ion energy fluxes

In [ ]:
plot_smoothed(
    [{"data": AUG_gem1, "style": '-.'}, {"data": AUG_gem2, "style": '-'}, {"data": AUG_tmp, "style": ':'}],
    [{"cols": Te_flux_cols, "label": 'electron energy flux density [W m^{-2}]', "limit": (1e2,0.7e5)},
     {"cols": Ti_flux_cols, "label": 'ion energy flux density [W m^{-2}]', "limit": (1e2,1.2e5)}])

In [ ]:
plot_smoothed(
    [{"data": AUG_gem2, "style": '-'}, {"data": AUG_tmp, "style": ':'}],
    [{"cols": Te_flux_cols, "label": 'electron energy flux density [W m^{-2}]', "limit": (0e4,0.7e5)},
     {"cols": Ti_flux_cols, "label": 'ion energy flux density [W m^{-2}]', "limit": (0e4,1.2e5)}],
    logy=False)

## Flux tube Te & Ti

In [ ]:
plot_unsmoothed(
    [{"data": AUG_gem1, "style": '-.'}, {"data": AUG_gem2, "style": ':'}],
    [{"cols": Te_cols, "label": 'electron temperature [eV]', "limit": (200,3500)},
     {"cols": Ti_cols, "label": 'ion temperature [eV]', "limit": (200,2500)}])

In [ ]:
plot_smoothed(
    [{"data": AUG_gem1, "style": '-.'}, {"data": AUG_gem2, "style": ':'}],
    [{"cols": Te_cols, "label": 'electron temperature [eV]', "limit": (200,3500)},
     {"cols": Ti_cols, "label": 'ion temperature [eV]', "limit": (200,2500)}])

## Flux tube dTe & dTi

In [ ]:
plot_unsmoothed(
    [{"data": AUG_gem1, "style": '-.'}, {"data": AUG_gem2, "style": ':'}],
    [{"cols": dTe_cols, "label": 'electron temperature gradient [eV/m]', "limit": (-8000,0)},
     {"cols": dTi_cols, "label": 'ion temperature gradient [eV/m]', "limit": (-5000,0)}],
    logy=False)

In [ ]:
plot_smoothed(
    [{"data": AUG_gem1, "style": '-.'}, {"data": AUG_gem2, "style": ':'}],
    [{"cols": dTe_cols, "label": 'electron temperature gradient [eV/m]', "limit": (-8000,0)},
     {"cols": dTi_cols, "label": 'ion temperature gradient [eV/m]', "limit": (-5000,0)}],
    logy=False)

## Flux tube ne & dne (not evolved so should be pretty much constant)

In [ ]:
plot_unsmoothed(
    [{"data": AUG_gem1, "style": '-.'}, {"data": AUG_gem2, "style": ':'}],
    [{"cols": ne_cols, "label": 'electron density [m^{-3}]', "limit": (3.5e19,4.5e19)},
     {"cols": dne_cols, "label": 'electron density gradient [m^{-3}/m]', "limit": (-0.65e19,-0.45e19)}],
    logy=False)

## Flux tube ne fluxes

In [ ]:
plot_unsmoothed(
    [{"data": AUG_gem1, "style": '-.'}, {"data": AUG_gem2, "style": ':'}],
    [{"cols": ne_flux_cols, "label": 'electron particle flux [s^{-1}]', "limit": (-2e20,2e20)},
     {"cols": ne_flux_cols, "label": 'electron particle flux [s^{-1}]', "limit": (-1e20,1e20)}],
    logy=False)

In [ ]:
plot_smoothed(
    [{"data": AUG_gem1, "style": '-.'}, {"data": AUG_gem2, "style": ':'}],
    [{"cols": ne_flux_cols, "label": 'electron particle flux [s^{-1}]', "limit": (-2e20,2e20)},
     {"cols": ne_flux_cols, "label": 'electron particle flux [s^{-1}]', "limit": (-1e20,1e20)}],
    logy=False)

## Summed energy fluxes

In [ ]:
plot_smoothed(
    [{"data": AUG_gem2, "style": '-'}, {"data": AUG_tmp, "style": ':'}],
    [{"cols": E_flux_cols, "label": 'summed energy flux density [W m^{-2}]', "limit": (0e4,2.0e5)}],
    logy=False, figsize=(24,16))

In [ ]:
import utility, importlib
importlib.reload(utility)
from utility import _ewma, plot_smoothed, plot_unsmoothed

# The End